# ================================================
# PIPELINE DE MACHINE LEARNING COM PLN E VECTORDB
# ================================================

## INSTALAÇÃO DE DEPENDÊNCIAS

In [18]:
# !pip install sentence-transformers
# !pip install chromadb
# !pip install pdfplumber
# !pip install nltk
# !pip install spacy
# !python -m spacy download pt_core_news_sm
# !pip install langchain_text_splitters

## IMPORTS

In [ ]:
import json
import csv
import re
import spacy
import nltk
import chromadb
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

#nltk.download('stopwords') # rodar apenas uma vez
#nltk.download("punkt")
#nltk.download("punkt_tab")

## ETAPAS

### 1. CARREGAMENTO DE ARQUIVOS

#### Definição das funções

In [20]:
def carregar_pdf(caminho_arquivo):
    """Carrega e extrai texto de um arquivo PDF"""
    import pdfplumber
    texto_completo = []
    with pdfplumber.open(caminho_arquivo) as pdf:
        for pagina in pdf.pages:
            texto = pagina.extract_text()
            if texto:
                texto_completo.append(texto)
    return "\n".join(texto_completo)

def carregar_csv(caminho_arquivo):
    """Carrega e extrai texto de um arquivo CSV"""
    textos = []
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for linha in reader:
            texto = " ".join([str(v) for v in linha.values()])
            textos.append(texto)
    return "\n".join(textos)

def carregar_json(caminho_arquivo):
    """Carrega e extrai texto de um arquivo JSON"""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        dados = json.load(f)

    textos = []

    def extrair_texto_recursivo(obj):
        if isinstance(obj, dict):
            for valor in obj.values():
                extrair_texto_recursivo(valor)
        elif isinstance(obj, list):
            for item in obj:
                extrair_texto_recursivo(item)
        elif isinstance(obj, str):
            textos.append(obj)

    extrair_texto_recursivo(dados)
    return "\n".join(textos)

def carregar_txt(caminho_arquivo):
    """Carrega e extrai texto de um arquivo TXT"""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return f.read()

def carregar_arquivo(caminho_arquivo):
    """Função universal para carregar arquivos PDF, CSV ou JSON"""
    extensao = caminho_arquivo.split('.')[-1].lower()

    if extensao == 'pdf':
        return carregar_pdf(caminho_arquivo)
    elif extensao == 'csv':
        return carregar_csv(caminho_arquivo)
    elif extensao == 'json':
        return carregar_json(caminho_arquivo)
    elif extensao == 'txt':
        return carregar_txt(caminho_arquivo)
    else:
        raise ValueError(f"Formato não suportado: {extensao}")

#### Execução

In [44]:
texto = ""
texto = carregar_arquivo('Existencialismo-em-Dark-Souls.pdf')
print(texto)

COLÉGIO SHALLON
GABRIEL BORGES DE OLIVEIRA
EXISTENCIALISMO: DARK SOULS E O EXISTENCIALISMO
GOIÂNIA
2020
GABRIEL BORGES DE OLIVEIRA
EXISTENCIALISMO: DARK SOULS E O EXISTENCIALISMO
Trabalho para conclusão de trimestre na matéria de
filosofia para obtenção de nota máxima ou passível
de aprovação para o fechamento do trimestre
Professor orientador: Renato Borges
GOIÂNIA
2020
GABRIEL BORGES DE OLIVEIRA
EXISTENCIALISMO: DARK SOULS E O EXISTENCIALISMO
Trabalho para conclusão de trimestre na matéria de
filosofia para obtenção de nota máxima ou passível
de aprovação para o fechamento do trimestre
Professor orientador: Renato Borges
________________________________________
Professor Orientador
Goiânia, 26 de outubro de 2020
04
RESUMO
O objetivo desta dissertação é demonstrar o existencialismo dentro da obra interativa
de jogos eletrônicos chamada Dark Souls.
SUMÁRIO
RESUMO……………………………………………………………….04
INTRODUÇÃO……………………………………………………….. 06
DISSERTAÇÃO………………………………………………………. 07 ATÉ 16
 Capítulo 1…………

### 2. PROCESSAMENTO DE LINGUAGEM NATURAL (PLN)

#### Definição das Stop Words

In [22]:
nlp = spacy.load('pt_core_news_sm')

API_STOPWORDS_PT = set(stopwords.words('portuguese'))
API_STOPWORDS_PT.remove('não')  # Mantendo a palavra "não"

STOPWORDS_EN = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 
                'has', 'he', 'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 
                'to', 'was', 'will', 'with', 'this', 'but', 'they', 'have', 'had'}

# Carregando apenas as stopwords em português

STOPWORDS_ALL = API_STOPWORDS_PT

#### Pré-Processamento de Texto

In [ ]:
def limpar_texto(texto):
    """Realiza limpeza básica do texto"""
    texto = texto.lower()
    texto = re.sub(r'\n\n', ' \n\n', texto) # manter quebras de parágrafo em caso de recursive chunking
    texto = re.sub(r'\n', ' \n', texto)
    texto = re.sub(r'\.', ' .', texto)
    texto = re.sub(r'[^a-z0-9áàâãéêíóôõúüçñ\s\n.]', ' ', texto) # ^ indica negação
    texto = re.sub(r'  +', ' ', texto) # multiplos espaços
    return texto.strip()

# Substituir acentos em caso de stemização

def tokenizar_nltk(texto):
    """Tokeniza o texto usando NLTK"""
    texto = word_tokenize(texto, language="english")
    return texto.split()

def tokenizar_spacy(texto):
    """Tokeniza o texto usando spaCy"""
    doc = nlp(texto)
    return [token for token in doc]

def remover_stopwords(tokens, stopwords=STOPWORDS_ALL):
    """Remove stopwords em português e inglês"""
    return [token for token in tokens if token.text not in stopwords]

def remover_stopwords_nltk(tokens, stopwords=STOPWORDS_ALL):
    """Remove stopwords usando NLTK"""
    return [token for token in tokens if token not in stopwords]

def lematizar_texto(tokens):
    """Pega a palavra lema de cada token no texto"""
    return [token.lemma_ for token in tokens]

def processar_texto_pln(texto):
    """Pipeline completo de PLN"""
    texto_limpo = limpar_texto(texto)
    palavras = tokenizar_spacy(texto_limpo)
    palavras_sem_stopwords = remover_stopwords(palavras)
    palavras_lemma = lematizar_texto(palavras_sem_stopwords)
    return ' '.join(palavras_lemma)

#### Execução

In [45]:
texto_processado = processar_texto_pln(texto)
print(texto_processado)

colégio shallon 
 Gabriel Borges oliveira 
 existencialismo Dark souls existencialismo 
 goiâniar 
 2020 
 Gabriel Borges oliveira 
 existencialismo Dark souls existencialismo 
 trabalho conclusão trimestre matéria 
 filosofia obtenção nota grande passível 
 aprovação fechamento trimestre 
 professor orientador Renato Borges 
 goiâniar 
 2020 
 Gabriel Borges oliveira 
 existencialismo Dark souls existencialismo 
 trabalho conclusão trimestre matéria 
 filosofia obtenção nota grande passível 
 aprovação fechamento trimestre 
 professor orientador Renato Borges 
 
 professor orientador 
 goiânia 26 outubro 2020 
 04 
 resumo 
 objetivo de este dissertação demonstrar existencialismo dentro obra interativo 
 jogo eletrônico chamar Dark souls . 
 sumário 
 resumo .04 
 introdução . . 06 
 dissertação . 07 16 
  capítulo 1 . 07 
  capítulo 2 . 08 10 
  capítulo 2 .1 . . 11 
  capítulo 2 .2 . . 12 
  capítulo 3 . 13 16 
 consideração final 17 
 referência bibliografio 18 
 06 
 introdução 
 

### 3. CHUNKING COM OVERLAP

#### Tipos relevantes de chunking

##### Fixed-Length Chunking

Divide o texto em pedaços com número fixo de caracteres ou tokens. É simples de implementar e previsível, porém pode cortar informações importantes.

In [25]:
# EXEMPLO:

# fixed_splitter = CharacterTextSplitter(
#     separator=".",
#     chunk_size=150,
#     chunk_overlap=20
# )

# chunks = fixed_splitter.split_text(texto_processado)
# print("n_chunks: ", len(chunks))
# print(chunks)

##### Recursive Character Chunking

Divide o texto progressivamente, mantendo mantendo estruturas naturais (parágrafos, frases), a melhor abordagem na maioria dos casos, superior ao fixed-length

In [58]:
# EXEMPLO:

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
    #separators=["\n\n", ".", "\n", " ", ""] # '.' tem mais valor semântico que '\n' pelo modo como o arquivo(pdf) é carregado
    separators=["\n\n", "\n", ".", " ", ""] # '.' tem mais valor semântico que '\n' pelo modo como o arquivo(pdf) é carregado
)

chunks = recursive_splitter.split_text(texto_processado)

for index, chunk in enumerate(chunks):
    chunks[index] = re.sub(r' \n', '', chunks[index])
    chunks[index] = re.sub(r'\. ', '', chunks[index])
    chunks[index] = re.sub(r' \.', '', chunks[index])
    #chunk = re.sub(r'\,', '', chunk)

print("n_chunks: ", len(chunks))
print(chunks)

n_chunks:  143
['colégio shallon Gabriel Borges oliveira existencialismo Dark souls existencialismo goiâniar 2020 Gabriel Borges oliveira', 'existencialismo Dark souls existencialismo trabalho conclusão trimestre matéria filosofia obtenção nota grande passível', 'aprovação fechamento trimestre professor orientador Renato Borges goiâniar 2020 Gabriel Borges oliveira', 'existencialismo Dark souls existencialismo trabalho conclusão trimestre matéria filosofia obtenção nota grande passível', 'aprovação fechamento trimestre professor orientador Renato Borges professor orientador goiânia 26 outubro 2020 04 resumo', '04 resumo objetivo de este dissertação demonstrar existencialismo dentro obra interativo jogo eletrônico chamar Dark souls sumário', 'sumário resumo04 introdução 06 dissertação 07 16  capítulo 1 07  capítulo 2 08 10  capítulo 21 11', 'capítulo 22 12  capítulo 3 13 16 consideração final 17 referência bibliografio 18 06 introdução', '06 introdução em este trabalho apresentar histór

##### Document-Based Chunking

Divide o texto respeitando a estrutura hierárquica do documento, ótima abordagem para casos envolvendo textos com markdown, artigos científicos e acadêmicos.

In [27]:
# EXEMPLO COM MARKDOWN:

# headers_to_split_on = [
#     ("#", "Header 1"),
#     ("##", "Header 2"),
#     ("###", "Header 3"),
# ]

# markdown_splitter = MarkdownHeaderTextSplitter(
#     headers_to_split_on=headers_to_split_on
# )

# chunks_document = markdown_splitter.split_text(documento_markdown)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=150,
#     chunk_overlap=20
# )

# final_chunks = text_splitter.split_documents(chunks_document)

# chunks_document
# metadata definidos no headers_to_split_on -> chunk.metadata # {'Header 1': 'Introdução ao PLN'}
# conteúdo  # Texto ...

### 4. EMBEDDING

#### Definição da função

Modelo BERT é treinado especificamente para a Língua Portuguesa e é bom para tarefas de similaridade semântica

In [ ]:
from sentence_transformers import SentenceTransformer

ENCODE_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"
# ENCODE_MODEL = 'all-MiniLM-L6-v2' # leve

modelo = SentenceTransformer(ENCODE_MODEL)

def criar_embeddings(chunks):
    """Cria embeddings para os chunks"""
    embeddings = modelo.encode(chunks, show_progress_bar=True)
    return embeddings

d:\Programação\machine-learning-study\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Gustavo e Junior\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


#### Execução

In [59]:
embedding = criar_embeddings(chunks)
print("Dimensão do embedding: ", embedding.shape)
print(embedding[:5])

Batches: 100%|██████████| 5/5 [00:20<00:00,  4.02s/it]

Dimensão do embedding:  (143, 384)
[[-0.07327388  0.10644147 -0.04687437 ... -0.12790535 -0.2797029
  -0.18030216]
 [-0.06255776  0.263058   -0.19158626 ... -0.23012151 -0.17705351
  -0.05989986]
 [-0.04214865  0.05695643  0.03917027 ... -0.11421223 -0.10755508
  -0.07304869]
 [-0.06255776  0.263058   -0.19158626 ... -0.23012151 -0.17705351
  -0.05989986]
 [-0.14981723  0.1575963   0.04524947 ... -0.07464445 -0.20204502
   0.00439715]]


### 5. VECTOR DATABASE

#### Inicializando cliente

In [62]:
client = chromadb.PersistentClient(path='./chroma_db')
    
collection = client.get_or_create_collection(name='documentos_ml', embedding_function=None)

print(f"✓ ChromaDB inicializado: {collection.count()} documentos na coleção")

✓ ChromaDB inicializado: 0 documentos na coleção


#### Apaga a collection

In [61]:
if collection.count() > 0:
    client.delete_collection(name='documentos_ml')

#### Adicionar ao chroma

In [63]:
# Criar ids únicos
ids = [f"chunk_{i}" for i in range(len(chunks))]

collection.add(
    ids=ids,
    documents=chunks,
    embeddings=embedding.tolist()  # converter para lista
)

### 6. BUSCA VETORIAL (QUERY)

In [ ]:
questions = ["Quantos filhos Gwyn tem?"]

for question in questions:
    question_embedding = modelo.encode([question])
    resultados = collection.query(
        query_embeddings=question_embedding.tolist(),
        n_results=3
    )
    print(f"\nPergunta: {question}")
    for doc, score in zip(resultados['documents'][0], resultados['distances'][0]):
        print(f" - Documento: {doc} (Score: {score})")


Pergunta: Quantos filhos Gwyn tem?
 - Documento: ornesteinr quatro cavaleiro gwyn smough carrasco reino destacado tarefa auxiliar gwyndolin proteger princesa Gwynevere (Score: 18.342031478881836)
 - Documento: escuridão não iniciar antes partir gwyn ordenar filho novo gwyndolin proteger cidade anor lor gwyndolin criar garota conta grande afinidade (Score: 20.558917999267578)
 - Documento: gwyn bruxa izalith filha nito primeiro morto seath descamado reino lordran erguer gwyn passar tratar Deus (Score: 20.72258186340332)
 - Documento: após algum ano batalha contra dragão gwyn 3 filho começar perceber primeiro chama começar apagar dois anomalia (Score: 23.18965721130371)
 - Documento: morto bruxa izalith filha caos gwyn senhor luz sol cavaleiro fiel pigmeu furtivo tão facilmente esquecer (Score: 24.62253189086914)
 - Documento: decorrer tempo gwyn perder contato bruxa izalith desolar resolver sacrificar chama estender fogo algum centena ano (Score: 25.05978012084961)
 - Documento: usar i